In [2]:
import math
import os
import subprocess
import time
import scipy
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import pandas as pd
import itertools
import pickle
from tabulate import tabulate
from g4beam import *
from tqdm import *
from skopt import gp_minimize
import os
from scan import *

In [3]:

print(os.getcwd())  # Print current working directory
print(os.listdir())


/Users/ryanmichaud/Downloads/final_cooling
['G4_PhaseRotation.g4bl', 'in_1738987901_0295.txt', 'out_1723476769_78412.txt', 'temp', 'test_particles_before.txt', '.DS_Store', 'in_1739340747_350017.txt', 'G4_RFCavity.g4bl', 'G4_QuadTest.g4bl', 'in_1723476769_78412.txt', '__pycache__', 'out_1738987901_0295.txt', 'g4beam.py', 'g4bl.out', 'quadtest_particles_before.txt', 'logs', 'G4beamline-3.08-Intel.dmg', 'particles_before.txt', 'scan.py', 'PDFS', 'out_1739340747_350017.txt', 'G4_FinalCooling_auto.g4bl', 'particles_after.txt', 'optimization.ipynb', 'G4_SingleQuadTest.g4bl']


In [3]:

t_emit = 0.110  # mm
momentum = 100  # MeV/c
beta = 0.03  # m
alpha = 1.0     # dimensionless
l_emit = 1      # mm
pz_std = 0.8    # MeV/c
vd_dist = 24    # mm

pre_w1 = gen_distribution((beta, alpha, t_emit, 0, 0), (beta, alpha, t_emit, 0, 0), momentum, pz_std, z_emit=l_emit, N=20000)
pre_w1["PDGid"] = -13
print_all_params(pre_w1)

def func(x):
    length, angle = x
    return emittances(cut_outliers(run_distribution(pre_w1, length, angle, vd_dist, axis=0)))[0]

start = time.time()
# Run optimization
optim_result = minimize(func, [7.5, 45], method="Nelder-Mead", bounds=((1, 10), (30, 70)), options=dict(fatol=1e-6))

# Get results
w1_length, w1_angle = optim_result.x
print(f"Length = {w1_length:.2f} mm\nAngle = {w1_angle:.1f} deg")
print("Time spent:", time.time()-start)

# Runs a single case with the optimal parameters
post_w1 = run_distribution(pre_w1, w1_length, w1_angle, vd_dist, axis=0)
print_all_params(post_w1)
print_all_params(cut_outliers(post_w1))

-----------------------------
Twiss parameters for X
emit  = 0.11050448732601509 mm
beta  = 0.03064148800134685 m
gamma = 65.77687779978338 1/m
alpha = 1.0077208997872973
D     = 0.002031100657288097 m
D'    = -0.12239592524443405

Twiss parameters for Y
emit  = 0.10851523000587966 mm
beta  = 0.030011621949990658 m
gamma = 66.37659706270881 1/m
alpha = 0.9960267754296955
D     = -0.0015720420543265036 m
D'    = 0.11439995459120612

Z-emittance:  1.3946644388852798 mm
Z std: 193.89652070022467 mm
p std: 0.8029163695379552 MeV/c
Mean momentum: 99.99786763320154 MeV/c
-----------------------------
iter value
Looking for output file: 'temp/out_1744767887_701241_197.txt'
Contents of temp/: ['in_1741323295_816161_2.txt', 'in_1742488494_242504_197.txt', 'in_1742833268_280874_162.txt', 'out_1743648235_7723289_48.txt', 'out_1743980283_6022341_220.txt', 'out_1744767887_701241_197.txt']
Looking for output file: 'temp/out_1744767897_216646_191.txt'
Contents of temp/: ['in_1741323295_816161_2.txt',

In [17]:

"""# Function to optimize
def func(x):
    length, angle = x
    return emittances(cut_outliers(run_distribution(pre_w1, length, angle, vd_dist, axis=0)))[0]

start = time.time()
# Run optimization
optim_result = minimize(func, [7.5, 45], method="Nelder-Mead", bounds=((1, 10), (30, 70)), options=dict(fatol=1e-6))

# Get results
w1_length, w1_angle = optim_result.x
print(f"Length = {w1_length:.2f} mm\nAngle = {w1_angle:.1f} deg")
print("Time spent:", time.time()-start)

# Runs a single case with the optimal parameters
post_w1 = run_distribution(pre_w1, w1_length, w1_angle, vd_dist, axis=0)
print_all_params(post_w1)
print_all_params(cut_outliers(post_w1))
"""

iter value
Looking for output file: 'temp/out_1743986783_0298738_5.txt'
Contents of temp/: ['in_1741323295_816161_2.txt', 'in_1742488494_242504_197.txt', 'in_1742833268_280874_162.txt', 'out_1743648235_7723289_48.txt', 'out_1743980283_6022341_220.txt', 'out_1743986783_0298738_5.txt']
Looking for output file: 'temp/out_1743986790_6724322_92.txt'
Contents of temp/: ['in_1741323295_816161_2.txt', 'in_1742488494_242504_197.txt', 'in_1742833268_280874_162.txt', 'out_1743648235_7723289_48.txt', 'out_1743980283_6022341_220.txt', 'out_1743986790_6724322_92.txt']
Looking for output file: 'temp/out_1743986798_153429_250.txt'
Contents of temp/: ['in_1741323295_816161_2.txt', 'in_1742488494_242504_197.txt', 'in_1742833268_280874_162.txt', 'out_1743648235_7723289_48.txt', 'out_1743980283_6022341_220.txt', 'out_1743986798_153429_250.txt']
Looking for output file: 'temp/out_1743986805_728609_9.txt'
Contents of temp/: ['in_1741323295_816161_2.txt', 'in_1742488494_242504_197.txt', 'in_1742833268_280874

In [4]:
post_w1 = run_distribution(pre_w1, w1_length, w1_angle, vd_dist, axis=0)
print_all_params(post_w1)
print_all_params(cut_outliers(post_w1))

Looking for output file: 'temp/out_1744768655_985591_20.txt'
Contents of temp/: ['in_1741323295_816161_2.txt', 'in_1742488494_242504_197.txt', 'in_1742833268_280874_162.txt', 'out_1743648235_7723289_48.txt', 'out_1743980283_6022341_220.txt', 'out_1744768655_985591_20.txt']
-----------------------------
Twiss parameters for X
emit  = 0.036739906490308116 mm
beta  = 0.04119371357471668 m
gamma = 228.64572688136207 1/m
alpha = -2.9015110861814275
D     = 0.016575894644228744 m
D'    = -0.18114685560747767

Twiss parameters for Y
emit  = 0.11247541492117878 mm
beta  = 0.022575407067961895 m
gamma = 75.37337535298171 1/m
alpha = -0.8376064891581536
D     = -0.00015498449012749192 m
D'    = 0.005634709764035569

Z-emittance:  7.073039726405771 mm
Z std: 181.85975384811562 mm
p std: 6.165030933106956 MeV/c
Mean momentum: 89.3795338613712 MeV/c
-----------------------------
-----------------------------
Twiss parameters for X
emit  = 0.03129797487914597 mm
beta  = 0.045237379867372335 m
gamma 

In [7]:
# Calculate dispersion correction
post_correct = remove_dispersion(post_w1)
print_all_params(post_correct)



# Ignore transverse momentums
no_transverse = remove_transverse(post_correct)

# Reverse transverse momentums in saved copy
reverse_transverse = post_correct.copy(deep=True)
reverse_transverse["Px"] *= -1
reverse_transverse["Py"] *= -1

-----------------------------
Twiss parameters for X
emit  = 0.03667049526923239 mm
beta  = 0.04126793545818529 m
gamma = 228.9727607575586 1/m
alpha = -2.906756459118209
D     = -4.1620865231892804e-05 m
D'    = -0.005941678003986255

Twiss parameters for Y
emit  = 0.11246775506850358 mm
beta  = 0.022574996135118656 m
gamma = 75.37524096231152 1/m
alpha = -0.8376131406609021
D     = 2.794531063654729e-06 m
D'    = 0.0001276731440558008

Z-emittance:  7.072353505785125 mm
Z std: 181.85138210685156 mm
p std: 6.1570742934087725 MeV/c
Mean momentum: 89.37182116266017 MeV/c
-----------------------------


In [8]:
drift_length = 16000
rf_length = 5153.756925848655
drift_to_start = drift_length-rf_length/2
post_drift = recenter_t(z_prop(no_transverse, drift_to_start))


In [9]:
drift_length = 16000
rf_freq = 0.025

start = time.time()
# Function to optimize
def func(x):
    rf_phase, rf_length, rf_grad = x
    drift_to_start = drift_length-rf_length/2
    post_drift = recenter_t(z_prop(no_transverse, drift_to_start))
    post_cavity = run_g4beam(post_drift, "G4_RFCavity.g4bl", RF_length=rf_length, frfcool=rf_freq, ficool=rf_phase, Vrfcool=rf_grad, nparticles=len(no_transverse))
    pre_w2 = recombine_transverse(post_cavity, reverse_transverse)
    return np.std(p_total(cut_pz(pre_w2)))

# Run optimization
optim_result = minimize(func, [0, 4700, 7], method="Nelder-Mead", bounds=((-90, 90), (2000, 6000), (1, 10)), options=dict(fatol=1e-6))

# Get results
rf_phase, rf_length, rf_grad = optim_result.x
print(f"Phase = {rf_phase:.2f} deg\nLength = {rf_length:.0f} mm\nGradient = {rf_grad:.2f} MV/m\nFrequency = {rf_freq*1000:.1f} MHz")
print("Time spent:", time.time()-start)

# Runs a single case with the optimal parameters and add the transverse back in
drift_to_start = drift_length-rf_length/2
post_drift = recenter_t(z_prop(no_transverse, drift_to_start))
post_cavity = cut_pz(recenter_t(run_g4beam(post_drift, "G4_RFCavity.g4bl", RF_length=rf_length, frfcool=rf_freq, ficool=rf_phase, Vrfcool=rf_grad)))
pre_w2 = recombine_transverse(post_cavity, reverse_transverse)
print_all_params(pre_w2)

iter value
Looking for output file: 'temp/out_1744768797_0096588_93.txt'
Contents of temp/: ['in_1741323295_816161_2.txt', 'in_1742488494_242504_197.txt', 'in_1742833268_280874_162.txt', 'out_1743648235_7723289_48.txt', 'out_1743980283_6022341_220.txt', 'out_1744768797_0096588_93.txt']
Looking for output file: 'temp/out_1744768821_546599_113.txt'
Contents of temp/: ['in_1741323295_816161_2.txt', 'in_1742488494_242504_197.txt', 'in_1742833268_280874_162.txt', 'out_1743648235_7723289_48.txt', 'out_1743980283_6022341_220.txt', 'out_1744768821_546599_113.txt']
Looking for output file: 'temp/out_1744768846_085276_244.txt'
Contents of temp/: ['in_1741323295_816161_2.txt', 'in_1742488494_242504_197.txt', 'in_1742833268_280874_162.txt', 'out_1743648235_7723289_48.txt', 'out_1743980283_6022341_220.txt', 'out_1744768846_085276_244.txt']
Looking for output file: 'temp/out_1744768872_0644858_112.txt'
Contents of temp/: ['in_1741323295_816161_2.txt', 'in_1742488494_242504_197.txt', 'in_1742833268_2

In [13]:
start = time.time()
# Function to optimize
def func(x):
    length, angle = x
    return emittances(cut_outliers(run_distribution(pre_w2, length, angle, vd_dist, axis=1)))[1]

# Run optimization
optim_result = minimize(func, [5.5, 45], method="Nelder-Mead", bounds=((1, 10), (30, 70)), options=dict(fatol=1e-6))

# Get results
w2_length, w2_angle = optim_result.x
print(f"Length = {w2_length:.2f} mm\nAngle = {w2_angle:.1f} deg")
print("Time spent:", time.time()-start)

# Runs a single case with the optimal parameters
post_w2 = run_distribution(pre_w2, w2_length, w2_angle, vd_dist, axis=1)
print_all_params(post_w2)

# Do cuts at this point
post_w2_cut = cut_outliers(post_w2)
print_all_params(post_w2_cut)


iter value
  35 3.59990e-02
Length = 5.45 mm
Angle = 49.1 deg
Time spent: 310.831650018692
-----------------------------
Twiss parameters for X
emit  = 0.03123213298253952 mm
beta  = 0.059199421222799205 m
gamma = 307.15439886572585 1/m
alpha = -4.145281973387068
D     = -0.00014362840257600308 m
D'    = -0.007416964880023907

Twiss parameters for Y
emit  = 0.03832996710142516 mm
beta  = 0.04560369817054347 m
gamma = 247.93107169482866 1/m
alpha = -3.2103852978529446
D     = 0.014682915991894887 m
D'    = 0.18081072565724715

Z-emittance:  26.100079237969208 mm
Z std: 621.5496238492692 mm
p std: 7.515990282140092 MeV/c
Mean momentum: 76.11803154583522 MeV/c
-----------------------------
-----------------------------
Twiss parameters for X
emit  = 0.03027673447071669 mm
beta  = 0.059264016150352754 m
gamma = 309.10938835697067 1/m
alpha = -4.161617928620211
D     = -0.00020940704194294236 m
D'    = -0.011020423493495784

Twiss parameters for Y
emit  = 0.03599895067473261 mm
beta  = 0.04

In [14]:
# Parameter list
PARAMS = [
    "t_emit",
    "momentum",
    "beta",
    "alpha",
    "l_emit",
    "pz_std",
    "vd_dist",
    "w1_length",
    "w1_angle",
    "w2_length",
    "w2_angle",
    "drift_length",
    "rf_freq",
    "rf_phase",
    "rf_length",
    "rf_grad"
]

In [ ]:
# Show parameters
{k:globals()[k] for k in PARAMS}

In [10]:
# Input parameters
parameters = {'t_emit': 0.110,
 'momentum': 100,
 'beta': 0.03,
 'alpha': 1,
 'l_emit': 1,
 'pz_std': 1,
 'vd_dist': 24,
 'w1_length': 9.20751061747799,
 'w1_angle': 49.78231333988419,
 'w2_length': 6.724887901827298,
 'w2_angle': 42.245718529695516,
 'drift_length': 16000,
 'rf_freq': 0.025,
 'rf_phase': 0.001987066319906211,
 'rf_length': 5153.756925848655,
 'rf_grad': 4.046563465382562}
globals().update(parameters)

In [ ]:
start = time.time()
# Run best case
pre_w1 = gen_distribution((beta, alpha, t_emit, 0, 0), (beta, alpha, t_emit, 0, 0), momentum, pz_std, z_emit=l_emit, N=50000)
pre_w1["PDGid"] = -13
print("Running first wedge")
post_w1 = run_distribution(pre_w1, w1_length, w1_angle, vd_dist, axis=0)
post_correct = remove_dispersion(post_w1)
reverse_transverse = post_correct.copy(deep=True)
reverse_transverse["Px"] *= -1
reverse_transverse["Py"] *= -1
drift_to_start = drift_length-rf_length/2
post_drift = recenter_t(z_prop(post_correct, drift_to_start))
no_transverse = remove_transverse(post_drift)
print("Running RF cavity")
post_cavity = cut_pz(recenter_t(run_g4beam(no_transverse, "G4_RFCavity.g4bl", RF_length=rf_length, frfcool=rf_freq, ficool=rf_phase, Vrfcool=rf_grad, nparticles=len(no_transverse))), tails=0.15)
print("Pre_w2 we are getting")
pre_w2 = recombine_transverse(post_cavity, reverse_transverse)
print_all_params(pre_w2)
print("Running second wedge")
post_w2 = run_distribution(pre_w2, w2_length, w2_angle, vd_dist, axis=1)
post_w2_cut = recenter_t(cut_outliers(post_w2))
print_all_params(post_w2_cut)

In [ ]:
print_all_params(post_correct)
print_all_params(pre_w2)

In [ ]:
# Make some phase space plots
PLOT_TARGETS = [pre_w1, post_w1,post_correct, post_drift, pre_w2, remove_dispersion(post_w2_cut)]
PLOT_TITLES = ["Initial distribution", "After 1st Wedges No Dipole","After first wedge dipole", "After drift", "After RF cavity and refocusing", "After second wedge"]
PLOT_AXES = [("x", "Px"), ("y", "Py"), ("t", "Pz")]
PLOT_LABELS = [("x (mm)", "Px (MeV/c)"), ("y (mm)", "Py (MeV/c)"), ("t (ns)", "Pz (MeV/c)")]
T_RANGES = [[-7.5, 7.5], [-30, 30]]
L_RANGES = [[-10, 10], [40, 110]]

fig, axes = plt.subplots(nrows = len(PLOT_AXES), ncols = len(PLOT_TARGETS), layout="constrained")
fig.set_size_inches(16, 8)
for i, df in enumerate(PLOT_TARGETS):
    axes[0][i].set_title(PLOT_TITLES[i])
    for j, ((xaxis, yaxis), (xlabel, ylabel)) in enumerate(zip(PLOT_AXES, PLOT_LABELS)):
        ax = axes[j][i]
        ax.hist2d(df[xaxis], df[yaxis], bins=50, range=L_RANGES if j == 2 else T_RANGES)
        ax.set_xlabel(xlabel)
        ax.set_ylabel(ylabel)
# fig.suptitle(f"temit={t_emit*1000:.0f} μm, σP={pz_std:.2f} MeV/c")

In [ ]:
print(post_w1)

PLOT_TARGETS = [pre_w1, post_w1, post_correct]
PLOT_TITLES = ["Initial distribution", "After First Wedge No Dipole" ,"After first wedge with Dipole"]
PLOT_AXES = [("x", "Px"), ("y", "Py"), ("t", "Pz")]
PLOT_LABELS = [("x (mm)", "Px (MeV/c)"), ("y (mm)", "Py (MeV/c)"), ("t (ns)", "Pz (MeV/c)")]
T_RANGES = [[-7.5, 7.5], [-30, 30]]
L_RANGES = [[-10, 10], [40, 110]]

fig, axes = plt.subplots(nrows = len(PLOT_AXES), ncols = len(PLOT_TARGETS), layout="constrained")
fig.set_size_inches(16, 8)
for i, df in enumerate(PLOT_TARGETS):
    axes[0][i].set_title(PLOT_TITLES[i])
    for j, ((xaxis, yaxis), (xlabel, ylabel)) in enumerate(zip(PLOT_AXES, PLOT_LABELS)):
        ax = axes[j][i]
        ax.hist2d(df[xaxis], df[yaxis], bins=50, range=L_RANGES if j == 2 else T_RANGES)
        ax.set_xlabel(xlabel)
        ax.set_ylabel(ylabel)
# fig.suptitle(f"temit={t_emit*1000:.0f} μm, σP={pz_std:.2f} MeV/c")

In [3]:
# Define parameters that I want
t_emit = 0.145
momentum = 100  # MeV/c
beta_x = 1.8    # m
beta_y = 0.4    # m
alpha = 0     # dimensionless
l_emit = 1      # mm
pz_std = 0.8

# Generate the distribution
particles_before = gen_distribution((beta_x, alpha, t_emit, 0, 0), (beta_y, alpha, t_emit, 0, 0), momentum, pz_std, z_emit=l_emit, N=50000)

# Run G4Beamline
single_cell_list = run_g4beam(particles_before, "G4_SingleQuadTest.g4bl",
                              multioutput=["0", "75", "475", "550", "625", "1025", "1100"],
                              fodo_grad=17.5341/3) # calculated using rigidity B_rho = 1/3 T*m

# Function to process list of distributions into table
def df_list_to_table(particles_list):
    z_list = [df.z[0] for df in particles_list]
    res = list()
    for z, df in zip(z_list, particles_list):
        (x_emit, x_beta, _, x_alpha, x_d, x_Dd), (y_emit, y_beta, _, y_alpha, y_d, y_Dd), z_emit = calc_all_params(cut_outliers(df))
        res.append([z, x_emit, x_beta, x_alpha, x_d, x_Dd, y_emit, y_beta, y_alpha, y_d, y_Dd, z_emit, len(df)])
    res = pd.DataFrame(res, columns=["z", "x_emit", "x_beta", "x_alpha", "x_d", "x_Dd", "y_emit", "y_beta", "y_alpha", "y_d", "y_Dd", "z_emit", "N"])
    res.z *= 0.001
    return res

# Run the function above
single_data = df_list_to_table(single_cell_list)

# Graph it
columns = ["x_beta", "x_alpha", "y_beta", "y_alpha"]
column_names = [r"$\beta_x$ (mm)", r"$\alpha_x$", r"$\beta_y$ (mm)", r"$\alpha_y$"]
fig, axes = plt.subplots(2, 2, figsize=(10, 8))
fig.suptitle("Periodic FODO Cell")
for ax, col, col_name in zip(axes.flat, columns, column_names):
    # ax.plot(madx_data["z"], madx_data[col], label="MAD-X", marker='o', linestyle='-', alpha=0.7)
    ax.plot(single_data["z"], single_data[col], label="G4Beamline", marker='s', linestyle='--', alpha=0.7)
    ax.set_xlabel("z (m)")
    ax.set_ylabel(col_name)
    ax.legend()
    ax.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

CalledProcessError: Command '['/Applications/G4beamline-3.08.app/Contents/MacOS/g4bl', 'G4_SingleQuadTest.g4bl', 'beamfile=temp/in_1745002353_193907_201.txt', 'outname=temp/out_1745002353_193907_201', 'nparticles=50000', 'fodo_grad=5.8447']' returned non-zero exit status 99.